<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/preprocess_translate_squad_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd

## Convert SQuAD JSON file to Pandas DataFrame

In [ ]:
!git clone https://github.com/pokjay/heb-squad

Cloning into 'heb-squad'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 64 (delta 19), reused 36 (delta 5), pack-reused 0
Unpacking objects: 100% (64/64), done.
Checking out files: 100% (21/21), done.


In [ ]:
with open('/content/heb-squad/data/original/train-v2.0.json') as f_handle:
    dataset = json.load(f_handle)

In [ ]:
def handle_answer(article, context, qa, answer_field):

  rows = []

  for answer in qa[answer_field]:

    q = qa['question']
    qid = qa['id']
    is_impossible = qa['is_impossible']
    answer_end = answer['answer_start'] + len(answer['text'])
                
    context_marked = context[:answer['answer_start']] + \
                    '[0123] ' + \
                    context[answer['answer_start'] : answer_end] + \
                    ' [4567]' \
                    + context[answer_end:]
                
    row = {'article': article['title'],
           'id': qid,
           'context': context_marked,
           'question': q,
           'answer': answer['text'],
           'answer_start': answer['answer_start'],
           'answer_end': answer_end,
           'is_impossible': int(is_impossible)}

    rows.append(row)
  
  return rows

In [ ]:
rows = []

for article in dataset['data']:
  for p in article['paragraphs']:
    # Some lines have \n, we remove them as they screw up translation
    context = p['context'].replace('\n', '')
    
    for qa in p['qas']:
      if 'answers' in qa:
        rows.extend(handle_answer(article, context, qa, 'answers'))
      if 'plausible_answers' in qa:
        rows.extend(handle_answer(article, context, qa, 'plausible_answers'))
      


In [ ]:
df = pd.DataFrame(rows).fillna("")
df.index = df.index.rename('idx')
df.shape

(130319, 8)

In [ ]:
df.head()

,article,id,context,question,answer,answer_start,answer_end,is_impossible
idx,,,,,,,,
0,Beyoncé,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269,286,0
1,Beyoncé,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207,226,0
2,Beyoncé,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526,530,0
3,Beyoncé,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166,180,0
4,Beyoncé,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276,286,0


In [ ]:
import numpy as np

idxs = np.round(np.array([df.shape[0], df.shape[0], df.shape[0], df.shape[0], df.shape[0]]) * np.array([0, 0.25, 0.5, 0.75, 1]))
idxs

array([     0.,  32580.,  65160.,  97739., 130319.])

In [ ]:
for i in range(len(idxs) - 1):
  partial_df = df.loc[idxs[i] : idxs[i+1]-1]
  # partial_df.drop(columns=['article']).to_excel(f'/content/heb-squad/data/intermediate/train-v2.0-{i}.xlsx', index=True)


## 1. Translate XLSX manually using Google Translate Document translator
## 2. Run code below processing the translated output

In [ ]:
translated_file_format = '/content/heb-squad/data/intermediate/heb-train-v2.0-{}.tsv'#'/Users/akeidar/temp/squad_train_{}_translated.tsv'

In [ ]:
dfs = []

for i in range(4):
    dfs.append(pd.read_csv(translated_file_format.format(i), sep='\t', quoting=3, index_col=0))#.dropna(subset=['context_marked'], how='any'))

df2 = pd.concat(dfs)

In [ ]:
df2['id'] = df2.id.str.strip()
df2['context'] = df2.context.str.strip()
df2['question'] = df2.question.str.strip()
df2['answer'] = df2.answer.str.strip().astype('str')

Take a quick look at what we got

In [ ]:
df2.shape

(130319, 7)

In [ ]:
df2.head(3)

,id,context,question,answer,answer_start,answer_end,is_impossible
idx,,,,,,,
0,56be85543aeaaa14008c9063,ביונסה ג'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON...,מתי ביונסה החלה להיות פופולרית?,בסוף שנות התשעים,269,286,0
1,56be85543aeaaa14008c9065,ביונסה ג'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON...,באילו תחומים ביונסה התחרה כשהתבגר?,שירה וריקודים,207,226,0
2,56be85543aeaaa14008c9066,ביונסה ג'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON...,מתי עזבה ביונסה את Destiny's Child והפכה לזמרת...,2003,526,530,0


### Validate IDs are correct, fix if needed

Check how many diffs we have

In [ ]:
pd.Series(df.id.values == df2.id.values).value_counts()

True     129721
False       598
dtype: int64

Print problematic IDs

In [ ]:
df2.loc[~(df.id.values == df2.id.values), 'id']

idx
518       56bec4de3aeaaa14008c93 להיות
904          56cbe2fd6d243a140015 ערוך
937           56cbeb396d243a140015 עדב
938           56cbeb396d243a140015 אדי
1028                                38
                      ...             
129776      57359bbcdc94161900571 איאה
129778         57359bbcdc94161900571ec
129779      57359bbcdc94161900571 הניח
129980      עמלת 5735c3d0dc94161900571
129986       5735c421dc94161900571 קפה
Name: id, Length: 598, dtype: object

Print original IDs

In [ ]:
df.loc[~(df.id.values == df2.id.values), 'id']

idx
518       56bec4de3aeaaa14008c93be
904       56cbe2fd6d243a140015edce
937       56cbeb396d243a140015edeb
938       56cbeb396d243a140015edec
1028      56cbfbdd6d243a140015ee38
                    ...           
129776    57359bbcdc94161900571eea
129778    57359bbcdc94161900571eec
129779    57359bbcdc94161900571eed
129980    5735c3d0dc94161900571fee
129986    5735c421dc94161900571ffe
Name: id, Length: 598, dtype: object

Replace problematic ids with original ids

In [ ]:
df2.loc[~(df.id.values == df2.id.values), 'id'] = df.loc[~(df.id.values == df2.id.values), 'id'].to_list()

In [ ]:
pd.Series(df.id.values == df2.id.values).value_counts()

True    130319
dtype: int64

### Add name of article for creating JSON later

In [ ]:
df2['article'] = df.article.to_list()
df2.article.value_counts() == df.article.value_counts()

Queen_Victoria                                    True
New_York_City                                     True
American_Idol                                     True
Beyoncé                                           True
Frédéric_Chopin                                   True
                                                  ... 
Order_of_the_British_Empire                       True
Letter_case                                       True
Race_and_ethnicity_in_the_United_States_Census    True
Grape                                             True
Pitch_(music)                                     True
Name: article, Length: 442, dtype: bool

Remove markers from context, and then print a sample to see if context looks good

In [ ]:
df2['context_marked'] = df2.context

In [ ]:
df2['context'] = df2.context_marked.str.replace('[0123] ', '', regex=False)
df2['context'] = df2.context.str.replace(' [4567]', '', regex=False)

In [ ]:
df2.iloc[10].context_marked, '', '', df2.iloc[10].context

('ביונסה ג\'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON-say) (נולדה ב -4 בספטמבר 1981) היא זמרת, כותבת שירים, מפיקה ושחקנית אמריקאית. היא נולדה וגדלה ביוסטון, טקסס, והופיעה בתחרויות שירה וריקוד בילדותה, ועלתה לתהילה בסוף שנות ה -90 כזמרת להקת הבנות R & B Destiny\'s Child. בניהולו של אביה, מתיו נואלס, הפכה הקבוצה לאחת מקבוצות הנערות הנמכרות ביותר בכל הזמנים. ההפסקה שלהם יצאה לאור אלבום הבכורה של ביונסה, [0123] Dangerously in Love [4567] (2003), שהקים אותה כאמנית סולו ברחבי העולם, זכתה בחמישה פרסי גראמי והציגה את סינגלי בילבורד הוט 100 מספר אחת "Crazy in Love "ו-" בייבי בוי ".',
 '',
 '',
 'ביונסה ג\'יזל נואלס-קרטר (/ biːˈjɒnseɪ/ bee-YON-say) (נולדה ב -4 בספטמבר 1981) היא זמרת, כותבת שירים, מפיקה ושחקנית אמריקאית. היא נולדה וגדלה ביוסטון, טקסס, והופיעה בתחרויות שירה וריקוד בילדותה, ועלתה לתהילה בסוף שנות ה -90 כזמרת להקת הבנות R & B Destiny\'s Child. בניהולו של אביה, מתיו נואלס, הפכה הקבוצה לאחת מקבוצות הנערות הנמכרות ביותר בכל הזמנים. ההפסקה שלהם יצאה לאור אלבום הבכורה של ביונסה, Dangerously 

Calculate the new start and end locations of the answer in the Hebrew context

In [ ]:
df2['answer_start_heb'] = df2.context_marked.str.find('[0123] ')
df2['answer_end_heb'] = df2.apply(lambda row: row.context_marked.find(' [4567]', int(row.answer_start_heb)) - len(' [4567]'), axis=1)

Validate how many rows are problematic and will be discarded

In [ ]:
print((df2.answer_start_heb >= 0).value_counts())
print()
print((df2.answer_end_heb >= 0).value_counts())

True     114467
False     15852
Name: answer_start_heb, dtype: int64

True     112644
False     17675
Name: answer_end_heb, dtype: int64


In [ ]:
for row in df2.sample(30):
  df2.iloc[10].context[int(df2.iloc[10].answer_start_heb):int(df2.iloc[10].answer_end_heb)]

In [ ]:
for row in df2.sample(10).itertuples():
  print(row.question)
  print(row.context[int(row.answer_start_heb):int(row.answer_end_heb)])
  print()

באיזו רפובליקה הרובע האוטונומי של קראצ'אי לא היה בעבר חלק?


איזו מנהרה בארבעה שלבים שימשה באיזו מנהרה בארה"ב?
מנהרת אשד

מהו שיעור האוריינות של ניו דלהי על פי מפקד האוכלוסין 2011?
89.38%

מי היווה יותר ממחצית הסכום?
שווקים מתפתחים

באיזה תאריך מציינים את יום ההולדת הרשמי של המלכה?
הרביעי ביוני

כמה אישורים נתנה RIAA לביונסה?
64

מדוע המערכת של דיואי נחשבת אפריורית?
המציאות נקבעת על פי ניסיון העבר

היכן ממוקמת ארכיון האינטרנט?
סן פרנסיסקו

מה הופך את ממשק Windows 8 לקל לשימוש?
הוראות שמספקת מערכת ההפעלה לגבי הפונקציות שאליהן ניגשים באמצעות ממשק המשתמש

באיזו שנה פרסמה חברתו לשעבר של שוורצנגר ברברה אאוטלנד בייקר את זיכרונותיה?
2006



#### Try and fix answers ruined in translation
These are rows where the tags surrounding the answer was lost in translation, maeaning we cannot retrieve the correct indices of the answers from the Hebrew translation. We will try to use the translated answer to find the indices in the context.

In [ ]:
mis_ans_df = df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)]

In [ ]:
mis_ans_df.shape

(22895, 11)

In [ ]:
new_ans_start = mis_ans_df.apply(lambda row: row.context.find(row.answer), axis=1)
new_ans_end = mis_ans_df.apply(lambda row: -1 if (row.context.find(row.answer) == -1) else (row.context.find(row.answer) + len(row.answer)), axis=1)

new_ans_start_success = new_ans_start[new_ans_start > -1]
new_ans_end_success = new_ans_end[new_ans_end > -1]

df2.loc[new_ans_start_success.index, 'answer_start_heb'] = new_ans_start_success.to_list()
df2.loc[new_ans_end.index, 'answer_end_heb'] = new_ans_end.to_list()

In [ ]:
print((df2.answer_start_heb >= 0).value_counts())
print()
print((df2.answer_end_heb >= 0).value_counts())

True     121941
False      8378
Name: answer_start_heb, dtype: int64

True     117895
False     12424
Name: answer_end_heb, dtype: int64


Below we remove rows where the translation deleted the answer markers from the context.

In [ ]:
print('We will remove', df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)].id.nunique(), 'rows out of', df2.id.nunique())

We will remove 12424 rows out of 130319


In [ ]:
print('We will remove', df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)].id.nunique() / df2.id.nunique() * 100, 'of ids')

We will remove 9.533529262808955 of ids


In [ ]:
df2_clean = df2[(df2.answer_start_heb < df2.answer_end_heb) | (df2.answer_start_heb >= 0) | (df2.answer_end_heb >= 0)]

In [ ]:
df2_clean.shape

(121941, 11)

Now we can save the df to file, it is ready!

In [ ]:
!mkdir /content/heb-squad/data/final

mkdir: cannot create directory ‘/content/heb-squad/data/final’: File exists


In [ ]:
df2_clean.to_csv('/content/heb-squad/data/final/heb-train-v2.0.csv', index=False)

## Output as JSON in SQuAD format for easy training and evaluation using SQuAD scripts

In [ ]:
df = pd.read_csv('/content/heb-squad/data/final/heb-train-v2.0.csv')

In [ ]:
articles = list(df.article.unique())

In [ ]:
squad_dataset = {'version': 'v2.0', 'data': []}

for article in articles:
  article_dict = {'title': article, 'paragraphs': []}
  squad_dataset['data'].append(article_dict)
  article_df = df[df.article == article]
  paragraphs = list(article_df.context.unique())

  for p in paragraphs:
    paragraph_dict = {'context': p, 'qas': []}
    article_dict['paragraphs'].append(paragraph_dict)
    p_df = article_df[article_df.context == p]
    questions = list(p_df.question.unique())

    for q in questions:
      
      q_df = p_df[p_df.question == q]
      
      # Squad saves plausible answers of impossible questions differently
      if bool(q_df.iloc[0].is_impossible) is False:
        answer_field = 'answers'
      else:
        answer_field = 'plausible_answers'
      
      q_dict = {answer_field: []}
      
      paragraph_dict['qas'].append(q_dict)
      
      # All rows have same id and is_impssible, so take first
      # Transform is_impossible to boolean as we saved it as integer 
      q_dict['id'] = q_df.iloc[0].id
      q_dict['is_impossible'] = bool(q_df.iloc[0].is_impossible)
      q_dict['question'] = q

      # Only these values are saved in SQuAD data
      ans_df = q_df[['answer', 'answer_start_heb']]

      for idx, row in ans_df.iterrows():
        answer_text, answer_start = row
        ans_dict = {'text': answer_text, 'answer_start': answer_start}
        q_dict[answer_field].append(ans_dict)


In [ ]:
with open('/content/heb-squad/data/final/heb-train-v2.0.json', 'w', encoding='utf8') as f_handle:
    json.dump(squad_dataset, f_handle, ensure_ascii=False)